In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns

In [2]:
data = pd.read_csv("../data/Language Detection.csv")

In [3]:
data.Language.value_counts()

English       1385
French        1014
Spanish        819
Portugeese     739
Italian        698
Russian        692
Sweedish       676
Malayalam      594
Dutch          546
Arabic         536
Turkish        474
German         470
Tamil          469
Danish         428
Kannada        369
Greek          365
Hindi           63
Name: Language, dtype: int64

In [4]:
data.shape

(10337, 2)

In [5]:
# 'English'
data = data[data.Language.isin(['French', 'Russian'])]

In [6]:
data.shape

(1706, 2)

In [8]:
data.Language.value_counts()

French     1014
Russian     692
Name: Language, dtype: int64

In [17]:
def is_ru(col):
    return 1 if col == "Russian" else 0

In [21]:
def clean(text):
    # remove numbers in text
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text)
    # remove other symbols in text
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    return text

In [22]:
data['clean'] = data['Text'].apply(clean)

In [23]:
data['lang_code'] = data['Language'].apply(is_ru)

In [24]:
data.head()

,Text,Language,clean,lang_code
3250,Si vous disposez d'ouvrages ou d'articles de r...,French,si vous disposez d'ouvrages ou d'articles de r...,0
3251,Comment ajouter mes sources ?,French,comment ajouter mes sources,0
3252,Cette page ou section est en train d'être trad...,French,cette page ou section est en train d'être trad...,0
3253,Vous pouvez aider au développement de Wikipédi...,French,vous pouvez aider au développement de wikipédi...,0
3254,Le mot nature est un terme polysémique (c’est-...,French,le mot nature est un terme polysémique c’est-...,0


In [25]:
data.tail()

,Text,Language,clean,lang_code
6681,Однажды Мелли и Терри снова пришли встретиться...,Russian,однажды мелли и терри снова пришли встретиться...,1
6682,"О, привет, вы двое, так скажите нам Мэриан.",Russian,о привет вы двое так скажите нам мэриан.,1
6683,Как' теперь нарциссизм Мэриан рассказал им обо...,Russian,как' теперь нарциссизм мэриан рассказал им обо...,1
6684,"Думаю, она не хотела бы больше золотого хлеба,...",Russian,думаю она не хотела бы больше золотого хлеба ...,1
6685,Терри вы на самом деле немного похожи на этого...,Russian,терри вы на самом деле немного похожи на этого...,1


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [28]:
X = cv.fit_transform(data["clean"])

In [29]:
X.shape

(1706, 8603)

In [31]:
y = data["lang_code"]

In [36]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size = 0.20)

In [41]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [43]:
clf.fit(Xtrain, Ytrain)

MultinomialNB()

In [44]:
Ypred = clf.predict(Xtest)

In [45]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [46]:
accr = accuracy_score(Ytest, Ypred)
print(accr)

0.9941520467836257


In [48]:
print(classification_report(Ytest, Ypred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       198
           1       0.99      0.99      0.99       144

    accuracy                           0.99       342
   macro avg       0.99      0.99      0.99       342
weighted avg       0.99      0.99      0.99       342



In [49]:
cm = confusion_matrix(Ytest, Ypred)

In [51]:
# plt.figure(figsize=(15,10))
# sns.heatmap(cm, annot = True)
# plt.show()

In [89]:
languages = {0:"french", 1:"russian"}

In [77]:
x = cv.transform(["Eh bien, mon prince"])
print(clf.predict_proba(x))
print(clf.predict(x))

[[0.99851699 0.00148301]]
[0]


In [90]:
def predict(text):
    x = cv.transform([text])
    lang = clf.predict(x)
    probs = clf.predict_proba(x)
    print("Probability is: ", probs[0][0])
    print("The language is", languages[lang[0]])

In [91]:
predict("Eh bien, mon prince")

Probability is:  0.9985169899743216
The language is french


In [92]:
predict("Анна Павловна кашляла несколько дней, у нее был  грипп , как она говорила  (грипп  был тогда новое слово, употреблявшееся только редкими)")

Probability is:  1.8019460101600063e-12
The language is russian


In [93]:
predict("В записочках, разосланных утром с красным лакеем, было написано без различия во всех:   «Si vous n’avez rien de mieux à faire, M. le comte (или mon prince), et si la perspective de passer la soirée chez une pauvre malade ne vous effraye pas trop, je serai charmée de vous voir chez moi entre 7 et 10 heures.")

Probability is:  1.0
The language is french


In [94]:
predict("Je vois que je vous fais peur,  садитесь и рассказывайте.")

Probability is:  0.9999999992592592
The language is french


In [95]:
predict("— Вы весь вечер у меня, надеюсь?")

Probability is:  1.2138048461903313e-05
The language is russian


In [96]:
predict("— Le général Koutouzoff, — сказал Болконский, ударяя на последнем слоге  zoff , как француз, — a bien voulu de moi pour aide-de-camp...   — Et Lise, votre femme?")

Probability is:  0.9999999999971294
The language is french
